## Create tree directory

./Zhou_stressGranules <br>
├ code_clean <br>
├ data_int <br>
&nbsp;&nbsp;&nbsp;&nbsp;├ bam_subset <br>
&nbsp;&nbsp;&nbsp;&nbsp;└ sampleSheets <br>
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ sampleSheets_sgRNA_split <br>
└ output <br>
&nbsp;&nbsp;&nbsp;&nbsp;└ 1_mRNAseq_output_alignment <br>

The experimental design of the run:

| name | batch | condition | group |
|---|---|---|---|
| s1 | collA | totRNA_mock | All |
| s2 | collA | totRNA_mock | All |
| s3 | collB | totRNA_mock | All |
| s4 | collA | totRNA_arsenite | treatment1 |
| s5 | collB | totRNA_arsenite | treatment1 |
| s6 | collB | totRNA_arsenite | treatment1 |
| s7 | collB | totRNA_HS | treatment2 |
| s8 | collB | totRNA_HS | treatment2 |
| s9 | collB | totRNA_HS | treatment2 |
| s10 | collA | totRNA_NaCl | treatment3 |
| s11 | collB | totRNA_NaCl | treatment3 |
| s12 | collB | totRNA_NaCl | treatment3 |
| s13 | collA | totRNA_UV | treatment4 |
| s14 | collA | totRNA_UV | treatment4 |
| s15 | collB | totRNA_UV | treatment4 |
| s16 | collA | sgRNA_arsenite | treatment1 |
| s17 | collB | sgRNA_arsenite | treatment1 |
| s18 | collB | sgRNA_arsenite | treatment1 |
| s19 | collB | sgRNA_HS | treatment2 |
| s20 | collB | sgRNA_HS | treatment2 |
| s21 | collB | sgRNA_HS | treatment2 |
| s22 | collA | sgRNA_NaCl | treatment3 |
| s23 | collB | sgRNA_NaCl | treatment3 |
| s24 | collB | sgRNA_NaCl | treatment3 |
| s25 | collA | sgRNA_UV | treatment4 |
| s26 | collA | sgRNA_UV | treatment4 |
| s27 | collB | sgRNA_UV | treatment4 |

In order to run the following scripts, do the following:
- install the conda environment  
  - besides the normal conda channels, some other channels are needed to install all the packages; e.g. `-c mvdbeek -c mpi-ie`
- either: 
   - create the directory structure with the script provided - to match the 'sources.yaml'  or
   - modify the input directories and references that are saved in the 'sources.yaml' file

## 1) Processing bam files

### A) arrange the sample sheet for running snakePipes*

DESeq from mRNA-seq pipeline takes the 1st sample as control for all comparisons. But sample 'totRNA_mock' is control only for 'totRNA*' samples, not for the rest. 

a) first split the 'all_batch_corrected_sampleSheet.tsv' sample sheet to get the one for 'totRNA_*' samples.

b) In rest, for 'sgRNA_*' samples, the comparisons were done in separate sample sheets, that were obtained like this:

### B) Running mRNA-seq snakePipes
The basic comand is:

To run the mRNA-seq snakePipes with **'alignment,deepTools_qc' mode** to map reads to reference genome, then count the features using 'featureCount' and then doing DESeq analysis, start from 'bam' files (to save time) and run either:  
a) to run only one sample sheet such as the 'batch_totalRNA_corrected_sampleSheet.tsv', run script '1a_mRNAseq_align_1SampleSheet.sh'.

b) to run all the split sample sheets at once, run script '1b_mRNAseq_align_splitSampleSheets.sh' by calling it with the name of the main project folder:

To run the mRNA-seq snakePipes with **'alignment-free' mode** in order to do transcript-level expression quantification using 'salmon' and then transcript-level differential expression with 'Sleuth' and general-level differential expression with 'wasabi' and 'DESeq2', you have to start from 'fastq' files and run either:      

c) to run only one sample sheet such as the 'batch_totalRNA_corrected_sampleSheet.tsv', run script '1_mRNAseq_snakePipe_1SampleSheet.sh':  

d) to run all the split sample sheets at once, run script '1d_mRNAseq_tx_splitSampleSheets.sh':

## 2) Differential expression analysis  
Read counting is done during the mRNA-seq pipeline and count matrix is in 'featureCounts' directory, or in 'sleuth' for alignment-free. 

For DE analysis, the general html reports are in the 'DESeq2_' folders of the same mRNA-seq run. For plotting specific contrasts, use the '2a_run2218_DESeq.Rmd' script.

## 3) QC for fastq files  
To assess whether we have any contamination in the reads, use the '3_kraken_QC.sh' script and then plot the outcome with '3_run2218_kraken_contamination_figS3g.Rmd'.  

To assess the read distribution, use the scripts '3_RSeQC.sh' and '3_run2218_RSeQC_readDistribution_fig2e.Rmd' to run RSeQC assesment.

- the gene.gtf file was downloaded from:   
    ftp.sanger.ac.uk/pub/gencode/Gencode_human/release_27/gencode.v27.chr_patch_hapl_scaff.annotation.gtf.gz
    
- the gene.bed file was obtained with this code:  

## 5) Mutation Calling

Using GATK's HaplotypeCaller. For that, it is required to have:  
- an indexed reference genome; We have used GRCh38  

- a bed file with only the gene coordinates located on main chromosomes [1-23XY], with a padding of 10bp 

    a) create the bed file with the coordinates of the chromosomes

b) get only the coordinates for the genes from the gtf file

c) keep only the genes that are on main chromosomes:

d) add 10bp padding to the genes

Scripts:  
    - To prepare the files for HaplotypeCaller, run the script: **5_0prep_files.sh**  
    - To run HaplotypeCaller in normal mode, run the script: **5_1HaplotypeCaller_normal.sh**  
    - Necessary to get the number of bases (A,C,T,G) that were covered by the sequencing, run HaplotypeCaller in ERC mode: **5_2HaplotypeCaller_ERC.sh**  
    - To extract the SNVs from the HaplotypeCaller normal run, use script:  
    - To extract the number of bases covered by sequencing, use script:  